In [ ]:
!pip install qiskit
!pip install qiskit-Aer
!pip install qiskit_algorithms
!pip install matplotlib
!pip install pylatexenc
!pip install qiskit-ibm-runtime

# An Introduction to Algorithms using Qiskit

This introduction to algorithms using Qiskit provides a high-level overview to get started with the `qiskit_algorithms` library.

## How is the algorithm library structured?

`qiskit_algorithms` provides a number of [algorithms](https://qiskit-community.github.io/qiskit-algorithms/apidocs/algorithms.html) grouped by category, according to the task they can perform. For instance `Minimum Eigensolvers` to find the smallest eigen value of an operator, for example ground state energy of a chemistry Hamiltonian or a solution to an optimization problem when expressed as an Ising Hamiltonian. There are `Time Evolvers` for the time evolution of quantum systems and `Amplitude Estimators` for value estimation that can be used say in financial applications. The full set of categories can be seen in the documentation link above.

Algorithms are configurable, and part of the configuration will often be in the form of smaller building blocks. For instance `VQE`, the Variational Quantum Eigensolver, it takes a trial wavefunction, in the form of a `QuantumCircuit` and a classical optimizer among other things.

Let's take a look at an example to construct a VQE instance. Here, `TwoLocal` is the variational form (trial wavefunction), a parameterized circuit which can be varied, and `SLSQP` a classical optimizer. These are created as separate instances and passed to VQE when it is constructed. Trying, for example, a different classical optimizer, or variational form is simply a case of creating an instance of the one you want and passing it into VQE.

In [ ]:
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

num_qubits = 2
ansatz = TwoLocal(num_qubits, "ry", "cz")
optimizer = SLSQP(maxiter=1000)

Let's draw the ansatz so we can see it's a `QuantumCircuit` where θ\[0\] through θ\[7\] will be the parameters that are varied as VQE optimizer finds the minimum eigenvalue. We'll come back to the parameters later in a working example below.

In [ ]:
ansatz.decompose().draw("mpl")

But more is needed before we can run the algorithm so let's get to that next.

## How to run an algorithm?

Algorithms rely on the primitives to evaluate expectation values or sample circuits. The primitives can be based on a simulator or real device and can be used interchangeably in the algorithms, as they all implement the same interface.

In the VQE, we have to evaluate expectation values, so for example we can use the [qiskit.primitives.Estimator](https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.Estimator) which is shipped with the default Qiskit installation.

In [ ]:
from qiskit.primitives import Estimator

estimator = Estimator()

This estimator uses an exact, statevector simulation to evaluate the expectation values. We can also use a shot-based and noisy simulators or real backends instead. For more information of the simulators you can check out [Qiskit Aer](https://qiskit.github.io/qiskit-aer/apidocs/aer_primitives.html) and for the actual hardware [Qiskit IBM Runtime](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime).

With all the ingredients ready, we can now instantiate the VQE:

In [ ]:
from qiskit_algorithms import VQE

vqe = VQE(estimator, ansatz, optimizer)

Now we can call the [compute_mininum_eigenvalue()](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.VQE.html#qiskit_algorithms.VQE.compute_minimum_eigenvalue) method. The latter is the interface of choice for the application modules, such as Nature and Optimization, in order that they can work interchangeably with any algorithm within the specific category.

## A complete working example

Let's put what we have learned from above together and create a complete working example. VQE will find the minimum eigenvalue, i.e. minimum energy value of a Hamiltonian operator and hence we need such an operator for VQE to work with. Such an operator is given below. This was originally created by the Nature application module as the Hamiltonian for an H2 molecule at 0.735A interatomic distance. It's a sum of Pauli terms as below.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

So let's run VQE and print the result object it returns.

In [ ]:
result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

From the above result we can see the number of cost function (=energy) evaluations the optimizer took until it found the minimum eigenvalue of $\approx -1.85727$ which is the electronic ground state energy of the given H2 molecule. The optimal parameters of the ansatz can also be seen which are the values that were in the ansatz at the minimum value.

## Updating the primitive inside VQE

To close off let's also change the estimator primitive inside the a VQE. Maybe you're satisfied with the simulation results and now want to use a shot-based simulator, or run on hardware!

In this example we're changing to a shot-based estimator, still using Qiskit's reference primitive. However, you could replace the primitive by e.g. Qiskit Aer's estimator ([qiskit_aer.primitives.Estimator](https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.primitives.Estimator.html#qiskit_aer.primitives.Estimator)) or even a real backend ([qiskit_ibm_runtime.Estimator](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.Estimator)).

For noisy loss functions, the SPSA optimizer typically performs well, so we also update the optimizer. See also the [noisy VQE tutorial](03_vqe_simulation_with_noise.ipynb) for more details on shot-based and noisy simulations.

In [ ]:
from qiskit_algorithms.optimizers import SPSA

estimator = Estimator(options={"shots": 1000})

vqe.estimator = estimator
vqe.optimizer = SPSA(maxiter=100)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result)

Note: We do not fix the random seed in the simulators here, so re-running gives slightly varying results.

This concludes this introduction to algorithms using Qiskit. Please check out the other algorithm tutorials in this series for both broader as well as more in depth coverage of the algorithms.

# Advanced VQE Options

In the first algorithms tutorial, you learned how to set up a basic [VQE](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.VQE.html) algorithm. Now, you will see how to provide more advanced configuration parameters to explore the full range of the variational algorithms provided in this library: [VQE](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.VQE.html), [QAOA](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.QAOA.html) and [VQD](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.VQD.html) among others. In particular, this tutorial will cover how to set up a `callback` to monitor convergence and the use of custom `initial point`s and `gradient`s.

## Callback

Callback methods can be used to monitor optimization progress as the algorithm runs and converges to the minimum. The callback is invoked for each functional evaluation by the optimizer and provides the current optimizer value, evaluation count, current optimizer parameters etc. Note that, depending on the specific optimizer this may not be each iteration (step) of the optimizer, so for example if the optimizer is calling the cost function to compute a finite difference based gradient this will be visible via the callback.

This section demonstrates how to leverage callbacks in `VQE` to plot the convergence path to the ground state energy with a selected set of optimizers.

First, you need a qubit operator for VQE. For this example, you can use the same operator as used in the algorithms introduction, which was originally computed by [Qiskit Nature](https://qiskit-community.github.io/qiskit-nature/) for an H2 molecule.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

The next step is to instantiate the `Estimator` of choice for the evaluation of expectation values within `VQE`. For simplicity, you can select the [qiskit.primitives.Estimator](https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.Estimator) that comes as part of Qiskit.

In [ ]:
from qiskit.primitives import Estimator

estimator = Estimator()

You are now ready to compare a set of optimizers through the `VQE` callback. The minimum energy of the H2 Hamiltonian can be found quite easily, so the maximum number of iterations (`maxiter`) does not have to be very large. You can once again use `TwoLocal` as the selected trial wavefunction (i.e. ansatz).

In [ ]:
import numpy as np

from qiskit.circuit.library import TwoLocal

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP
from qiskit_algorithms.utils import algorithm_globals

# we will iterate over these different optimizers
optimizers = [COBYLA(maxiter=80), L_BFGS_B(maxiter=60), SLSQP(maxiter=60)]
converge_counts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

for i, optimizer in enumerate(optimizers):
    print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
    algorithm_globals.random_seed = 50
    ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

    counts = []
    values = []

    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)

    vqe = VQE(estimator, ansatz, optimizer, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H2_op)
    converge_counts[i] = np.asarray(counts)
    converge_vals[i] = np.asarray(values)

print("\rOptimization complete      ");

Now, from the callback data you stored, you can plot the energy value at each objective function call each optimizer makes. An optimizer using a finite difference method for computing gradient has that characteristic step-like plot where for a number of evaluations it is computing the value for close by points to establish a gradient (the close by points having very similar values whose difference cannot be seen on the scale of the graph here).

In [ ]:
import pylab

pylab.rcParams["figure.figsize"] = (12, 8)
for i, optimizer in enumerate(optimizers):
    pylab.plot(converge_counts[i], converge_vals[i], label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for various optimizers")
pylab.legend(loc="upper right");

Finally, since the above problem is still easily tractable classically, you can use [NumPyMinimumEigensolver](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.NumPyMinimumEigensolver.html) to compute a reference value for the solution.

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=H2_op)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

You can now plot the difference between the `VQE` solution and this exact reference value as the algorithm converges towards the minimum energy.

In [ ]:
pylab.rcParams["figure.figsize"] = (12, 8)
for i, optimizer in enumerate(optimizers):
    pylab.plot(
        converge_counts[i],
        abs(ref_value - converge_vals[i]),
        label=type(optimizer).__name__,
    )
pylab.xlabel("Eval count")
pylab.ylabel("Energy difference from solution reference value")
pylab.title("Energy convergence for various optimizers")
pylab.yscale("log")
pylab.legend(loc="upper right");

## Gradients

In the variational algorithms, if the provided optimizer uses a gradient-based technique, the default gradient method will be finite differences. However, these classes include an option to pass custom gradients via the `gradient` parameter, which can be any of the provided methods within the [gradient](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.gradients.html) framework, which fully supports the use of primitives. This section shows how to use custom gradients in the VQE workflow.

The first step is to initialize both the corresponding primitive and primitive gradient:

In [ ]:
from qiskit_algorithms.gradients import FiniteDiffEstimatorGradient

estimator = Estimator()
gradient = FiniteDiffEstimatorGradient(estimator, epsilon=0.01)

Now, you can inspect an SLSQP run using the [FiniteDiffEstimatorGradient](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.gradients.FiniteDiffEstimatorGradient.html) from above:

In [ ]:
algorithm_globals.random_seed = 50
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

optimizer = SLSQP(maxiter=100)

counts = []
values = []


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)


vqe = VQE(estimator, ansatz, optimizer, callback=store_intermediate_result, gradient=gradient)

result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(f"Value using Gradient: {result.eigenvalue.real:.5f}")

In [ ]:
pylab.rcParams["figure.figsize"] = (12, 8)
pylab.plot(counts, values, label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence using Gradient")
pylab.legend(loc="upper right");

## Initial point

By default, the optimization begins at a random point within the bounds defined by the ansatz. The `initial_point` option allows to override this point with a custom list of values that match the number of ansatz parameters.

You might wonder... *Why set a custom initial point?* Well, this option can come in handy if you have a guess for a reasonable starting point for the problem, or perhaps know information from a prior experiment.

To demonstrate this feature, let's look at the results from our previous VQE run:

In [ ]:
print(result)
cost_function_evals = result.cost_function_evals

Now, you can take the `optimal_point` from the above result and use it as the `initial_point` for a follow-up computation.

**Note:** `initial_point` is now a keyword-only argument of the `VQE` class (i.e, it must be set following the `keyword=value` syntax).

In [ ]:
initial_pt = result.optimal_point

estimator1 = Estimator()
gradient1 = FiniteDiffEstimatorGradient(estimator, epsilon=0.01)
ansatz1 = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
optimizer1 = SLSQP(maxiter=1000)

vqe1 = VQE(estimator1, ansatz1, optimizer1, gradient=gradient1, initial_point=initial_pt)
result1 = vqe1.compute_minimum_eigenvalue(operator=H2_op)
print(result1)

cost_function_evals1 = result1.cost_function_evals
print()

In [ ]:
print(
    f"cost_function_evals is {cost_function_evals1} with initial point versus {cost_function_evals} without it."
)

By looking at the `cost_function_evals` you can notice how the initial point helped the algorithm converge faster (in just 1 iteration, as we already provided the optimal solution).

This can be particularly useful in cases where we have two closely related problems, and the solution to one problem can be used to guess the other's. A good example might be plotting dissociation profiles in chemistry, where we change the inter-atomic distances of a molecule and compute its minimum eigenvalue for each distance. When the distance changes are small, we expect the solution to still be close to the prior one. Thus, a popular technique is to simply use the optimal point from one solution as the starting point for the next step. There also exist more complex techniques, where we can apply extrapolation to compute an initial position based on prior solution(s) rather than directly use the prior solution.

# VQE with Qiskit Aer Primitives

This notebook demonstrates how to leverage the [Qiskit Aer Primitives](https://qiskit.github.io/qiskit-aer/apidocs/aer_primitives.html) to run both noiseless and noisy simulations locally. Qiskit Aer not only allows you to define your own custom noise model, but also to easily create a noise model based on the properties of a real quantum device. This notebook will show an example of the latter, to illustrate the general workflow of running algorithms with local noisy simulators.

For further information on the Qiskit Aer noise model, you can consult the [Qiskit Aer documentation](https://qiskit.github.io/qiskit-aer/apidocs/aer_noise.html), as well the tutorial for [building noise models](https://qiskit.github.io/qiskit-aer/tutorials/3_building_noise_models.html).

The algorithm of choice is once again VQE, where the task consists on finding the minimum (ground state) energy of a Hamiltonian. As shown in previous tutorials, VQE takes in a qubit operator as input. Here, you will take a set of Pauli operators that were originally computed by Qiskit Nature for the H2 molecule, using the [SparsePauliOp](https://docs.quantum.ibm.com/api/qiskit/qiskit.quantum_info.SparsePauliOp) class.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

print(f"Number of qubits: {H2_op.num_qubits}")

As the above problem is still easily tractable classically, you can use `NumPyMinimumEigensolver` to compute a reference value to compare the results later.

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=H2_op)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

The following examples will all use the same ansatz and optimizer, defined as follows:

In [ ]:
# define ansatz and optimizer
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.optimizers import SPSA

iterations = 125
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
spsa = SPSA(maxiter=iterations)

## Performance *without* noise

Let's first run the `VQE` on the default Aer simulator without adding noise, with a fixed seed for the run and transpilation to obtain reproducible results. This result should be relatively close to the reference value from the exact computation.

In [ ]:
# define callback
# note: Re-run this cell to restart lists before training
counts = []
values = []


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

In [ ]:
# define Aer Estimator for noiseless statevector simulation
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer.primitives import Estimator as AerEstimator

seed = 170
algorithm_globals.random_seed = seed

noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)

In [ ]:
# instantiate and run VQE
from qiskit_algorithms import VQE

vqe = VQE(noiseless_estimator, ansatz, optimizer=spsa, callback=store_intermediate_result)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)

print(f"VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}")
print(f"Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}")

You captured the energy values above during the convergence, so you can track the process in the graph below.

In [ ]:
import pylab

pylab.rcParams["figure.figsize"] = (12, 4)
pylab.plot(counts, values)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Convergence with no noise")

## Performance *with* noise

Now, let's add noise to our simulation. Here we create a fake device, using `GenericBackendV2`, which has a default noise model. As stated in the introduction, it is also possible to create custom noise models from scratch, but this task is beyond the scope of this notebook.

By default the fake device will have all to all coupling, so to make it more realistic, we will give it a coupling map (this map matches what the 5-qubit Vigo device had) Note: You can also use this coupling map as the entanglement map for the variational form if you choose to.

In [ ]:
from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import GenericBackendV2

coupling_map = [(0, 1), (1, 2), (2, 3), (3, 4)]
device = GenericBackendV2(num_qubits=5, coupling_map=coupling_map, seed=54)

noise_model = NoiseModel.from_backend(device)

print(noise_model)

Once the noise model is defined, you can run `VQE` using an Aer `Estimator`, where you can pass the noise model to the underlying simulator using the `backend_options` dictionary. Please note that this simulation will take longer than the noiseless one.

In [ ]:
noisy_estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "coupling_map": coupling_map,
        "noise_model": noise_model,
    },
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)

Instead of defining a new instance of the `VQE` class, you can now simply assign a new estimator to our previous `VQE` instance. As the callback method will be re-used, you will also need to re-start the `counts` and `values` variables to be able to plot the convergence graph later on.

In [ ]:
# re-start callback variables
counts = []
values = []

In [ ]:
vqe.estimator = noisy_estimator

result1 = vqe.compute_minimum_eigenvalue(operator=H2_op)

print(f"VQE on Aer qasm simulator (with noise): {result1.eigenvalue.real:.5f}")
print(f"Delta from reference energy value is {(result1.eigenvalue.real - ref_value):.5f}")

In [ ]:
if counts or values:
    pylab.rcParams["figure.figsize"] = (12, 4)
    pylab.plot(counts, values)
    pylab.xlabel("Eval count")
    pylab.ylabel("Energy")
    pylab.title("Convergence with noise")

## Summary


In this tutorial, you compared three calculations for the H2 molecule ground state.  First, you produced a reference value using a classical minimum eigensolver. Then, you proceeded to run `VQE` using the Qiskit Aer `Estimator` with 1024 shots. Finally, you extracted a noise model from a backend and used it to define a new `Estimator` for noisy simulations. The results are:

In [ ]:
print(f"Reference value: {ref_value:.5f}")
print(f"VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}")
print(f"VQE on Aer qasm simulator (with noise): {result1.eigenvalue.real:.5f}")

You can notice that, while the noiseless simulation's result is closer to the exact reference value, there is still some difference. This is due to the sampling noise, introduced by limiting the number of shots to 1024. A larger number of shots would decrease this sampling error and close the gap between these two values.

As for the noise introduced by real devices (or simulated noise models), it could be tackled through a wide variety of error mitigation techniques. The [Qiskit Runtime Primitives](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime) have enabled error mitigation through the `resilience_level` option. This option is currently available for remote simulators and real backends accessed via the Runtime Primitives, you can consult [this documentation](https://docs.quantum.ibm.com/run/configure-error-mitigation) for further information.

# Variational Quantum Deflation (VQD) Algorithm

This notebook demonstrates how to use our implementation of the [Variational Quantum Deflation (VQD)](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.VQD.html) algorithm for computing higher energy states of a Hamiltonian, as introduced in this [reference paper](https://arxiv.org/abs/1805.08138).

## Introduction

VQD is a quantum algorithm that uses a variational technique to find the *k* eigenvalues of the Hamiltonian *H* of a given system.

The algorithm computes excited state energies of generalized hamiltonians by optimizing over a modified cost function. Each successive eigenvalue is calculated iteratively by introducing an overlap term with all the previously computed eigenstates that must be minimized. This ensures that higher energy eigenstates are found.

## Complete working example for VQD

The first step of the VQD workflow is to create a qubit operator, ansatz and optimizer. For this example, you can use the H2 molecule, which should already look familiar if you have completed the previous VQE tutorials:

In [ ]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

You can set up, for example, a `TwoLocal` ansatz with two qubits, and choose `SLSQP` as the optimization method.

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.optimizers import SLSQP

ansatz = TwoLocal(2, rotation_blocks=["ry", "rz"], entanglement_blocks="cz", reps=1)

optimizer = SLSQP()
ansatz.decompose().draw("mpl")

The next step of the workflow is to define the required primitives for running `VQD`. This algorithm requires two different primitive instances: one `Estimator` for computing the expectation values for the "VQE part" of the algorithm, and one `Sampler`. The sampler will be passed along to the `StateFidelity` subroutine that will be used to compute the cost for higher energy states. There are several methods that you can use to compute state fidelities, but to keep things simple, you can use the `ComputeUncompute` method already available in `qiskit_algorithm.state_fidelities`.

In [ ]:
from qiskit.primitives import Sampler, Estimator
from qiskit_algorithms.state_fidelities import ComputeUncompute

estimator = Estimator()
sampler = Sampler()
fidelity = ComputeUncompute(sampler)

In order to set up the VQD algorithm, it is important to define two additional inputs: the number of energy states to compute (`k`) and the `betas` defined in the original VQD paper. In this example, the number of states (`k`) will be set to three, which indicates that two excited states will be computed in addition to the ground state.

The `betas` balance the contribution of each overlap term to the cost function, and they are an optional argument in the `VQD` construction. If not set by the user, they can be auto-evaluated for input operators of type `SparsePauliOp`. Please note that if you want to set your own `betas`, you should provide a list of values of length `k`.

In [ ]:
k = 3
betas = [33, 33, 33]

You are almost ready to run the VQD algorithm, but let's define a callback first to store intermediate values:

In [ ]:
counts = []
values = []
steps = []


def callback(eval_count, params, value, meta, step):
    counts.append(eval_count)
    values.append(value)
    steps.append(step)

You can finally instantiate `VQD` and compute the eigenvalues for the chosen operator.

In [ ]:
from qiskit_algorithms import VQD

vqd = VQD(estimator, fidelity, ansatz, optimizer, k=k, betas=betas, callback=callback)
result = vqd.compute_eigenvalues(operator=H2_op)
vqd_values = result.eigenvalues

You can see the three state energies as part of the `VQD` result:

In [ ]:
print(vqd_values.real)

And we can use the values stored by the callback to plot the energy convergence for each state:

In [ ]:
import numpy as np
import pylab

pylab.rcParams["figure.figsize"] = (12, 8)

steps = np.asarray(steps)
counts = np.asarray(counts)
values = np.asarray(values)

for i in range(1, 4):
    _counts = counts[np.where(steps == i)]
    _values = values[np.where(steps == i)]
    pylab.plot(_counts, _values, label=f"State {i-1}")

pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for each computed state")
pylab.legend(loc="upper right");

This molecule can be solved exactly using the `NumPyEigensolver` class, which will give a reference value that you can compare with the `VQD` result:

In [ ]:
from qiskit_algorithms import NumPyEigensolver


exact_solver = NumPyEigensolver(k=3)
exact_result = exact_solver.compute_eigenvalues(H2_op)
ref_values = exact_result.eigenvalues

Let's see a comparison of the exact result with the previously computed `VQD` eigenvalues:

In [ ]:
print(f"Reference values: {ref_values}")
print(f"VQD values: {vqd_values.real}")

As you can see, the result from VQD matches the values from the exact solution, and extends VQE to also compute excited states.

# Send it after class

Now let's see if we can solve a bigger problem. In the following block, you can find how to implement one way a bigger basis set for the H$_2$ molecule (6-31g instead of STO-3G). This will create a bigger "PauliWord" that contains 8-long sentences. Here, we have used Jordan-Wigner mapping to map fermionic operators to qubits. We will talk about the exact details of "qubitization" later.

In [ ]:
!pip install qiskit_nature
!pip install --prefer-binary pyscf

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="6-31g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()
fermionic_op = es_problem.hamiltonian.second_q_op()



mapper = JordanWignerMapper()

qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)



Now modify the above VQD example to obtain the the ground state, and first two excited states. Do you see a difference?

In [ ]:
# Code me

As you can see, everything works for H$_2$, and going for a more detailed basis set didn't really improve anything. This is not always the case. Now try another molecule, such as Carbon Monoxide, using STO-3G, and see if going for a bigger basis set improves the result.

In [ ]:
# Code me